In [ ]:
from ultralytics import YOLO
import torch
from PIL import Image
from torchvision.transforms import v2
import torchvision.transforms as T
import glob
import os
import posixpath
import cv2
from numpy import asarray


In [ ]:
source = sorted(glob.glob('datasample/test_set_yolo/test*.jpg'),
                    key = lambda i: (len(posixpath.basename(i)), i))
print(source)

In [ ]:
model = YOLO("yolov8n.yaml") 
model = YOLO("./checkpoints/ckp_yolo_model.pt")


In [ ]:
result = model.predict(source, save=True, conf=0.5)

In [ ]:
i = range(len(source))

In [ ]:
source2 = sorted(glob.glob('datasample/test_set_yolo/*_label.jpg'),
                    key = lambda i: (len(posixpath.basename(i)), i))

In [ ]:
source2

In [ ]:
def add_margin(pil_img, top, right, bottom, left, color):
    width, height = pil_img.size
    new_width = width + right + left
    new_height = height + top + bottom
    result = Image.new(pil_img.mode, (new_width, new_height), color)
    result.paste(pil_img, (left, top))
    return result

In [ ]:
bbox1 = []
name = []

for n in i:
    boxes = result[n].boxes.xywhn.tolist()
    classes = result[n].boxes.cls.tolist()
    names = result[n].names
    confidences = result[n].boxes.conf.tolist()
    # k = result[n].boxes.keypoints.tolist()
    bbox1.append(boxes)
    name.append(confidences)

In [ ]:
bbox1

In [ ]:
bbox= []

for n in i:
    b = bbox1[n]
    c = b[0]
    bbox.append(c)

In [ ]:
from pathlib import Path
Path("./output/im/").mkdir(parents=True, exist_ok=True)
Path("./output/lab/").mkdir(parents=True, exist_ok=True)
Path("./output/txt").mkdir(parents=True, exist_ok=True)

for x, image, label in zip(bbox, source, source2):
    im = Image.open(image)
    lab = Image.open(label)
    tens = cv2.imread(image) 
    tens_lab = cv2.imread(label)
    
    image_width = tens.shape[1]
    image_height = tens.shape[0] 

    x_center = x[0]
    y_center = x[1]
    width = x[2]
    height = x[3]

    w = width * image_width
    h = height * image_height
    end_point0 = int((x_center * image_width) - (w/2))
    end_point1 = int((y_center * image_height) - (h/2))
    start_point0 = int(w + end_point0)
    start_point1 = int(h + end_point1)
 
    cropped_image1 = tens[end_point1:start_point1, end_point0:start_point0]
    cropped_label1 = tens_lab[end_point1:start_point1, end_point0:start_point0]
    cropped = Image.fromarray(cropped_image1)
    cropped_lab = Image.fromarray(cropped_label1)
    resize = v2.Resize(size=(1024, 512)) 
    cropped_image2 = resize(cropped)
    cropped_label2 = resize(cropped_lab)
    im_new = add_margin(cropped_image2, 0, 256, 0, 256, (0, 0, 0))
    lab_new = add_margin(cropped_label2, 0, 256, 0, 256, (0, 0, 0))
    cropped_image = asarray(im_new)
    cropped_label = asarray(lab_new)

    cropped_image_width = cropped_image1.shape[1]
    cropped_image_height = cropped_label1.shape[0] 
    
    x1 = end_point0
    y1 = end_point1
    x2 = image_width - start_point0
    y2 = image_height - start_point1

    base, ext = os.path.splitext(os.path.basename(image))  
    ext = ext.lower()  

    base2, ext2 = os.path.splitext(os.path.basename(label))  
    ext2 = ext2.lower() 
    
    cv2.imwrite('./output/im/'f"{base}{ext}", cropped_image)
    cv2.imwrite('./output/lab/'f"{base2}{ext2}", cropped_label)
    destination_path = os.path.join("./output/txt", f"{base}.txt")

    f = open(destination_path, "w")
    f.write(str(cropped_image_width) + " " + str(cropped_image_height) + " " + str(x1) + " " + str(y1) + " " + str(x2) + " " + str(y2))
    f.close()
